#Ingenieria de variables ⚙ 🚀

In [69]:
import pandas as pd
import numpy as np

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [70]:
#aca cargo un resumen de toda la ingenieria de variables hecha en el archivo llamado "Titanic_5" para que ambos
#df me queden iguales y asi poder poner a mi modelo a predecir
df = pd.read_csv('/content/drive/MyDrive/IES/Segundo semestre/Aprendizaje automatico I/Trabajo TITANIC Final/FINALASO TODO SUPER FINAL/cvs_predicciones.csv')
df

# Imputar edad
df['HasChildren'] = df['Parch'].apply(lambda x: 1 if x > 0 else 0)
age_means = df.groupby(['Sex', 'HasChildren', 'Pclass'])['Age'].mean().reset_index(name='AgeMean')
df['Age'] = df.apply(lambda row: age_means.loc[(age_means['Sex'] == row['Sex']) &
                                                (age_means['HasChildren'] == (1 if row['Parch'] > 0 else 0)) &
                                                (age_means['Pclass'] == row['Pclass']), 'AgeMean'].values[0]
                      if pd.isnull(row['Age']) else row['Age'], axis=1)
df.drop('HasChildren', axis=1, inplace=True)

#creo columnas que aporten mas informacione stando combinadas
df['Familia'] = df['SibSp'] + df['Parch'] + 1
df['Cuota_por_pasajero'] = df['Fare'] / df['Familia']
df['pclass'] = df['Pclass'].replace({1: 3, 3: 1})
df['Titulo'] = df['Name'].apply(lambda x: x.split(',')[1].split('.')[0].strip()).map({
    'Mr': 1, 'Mrs': 2, 'Miss': 3, 'Master': 4, 'Dr': 5, 'Rev': 6,
    'Col': 7, 'Major': 8, 'Sir': 9, 'Jonkheer': 10, 'Don': 11,
    'Lady': 12, 'Capt': 13, 'Countess': 14
}).fillna(0).astype(int)

# Mapear sex
df['Sex'] = df['Sex'].map({'male': 0, 'female': 1})
df['Pclass_Sex'] = df['Pclass'].astype(str) + df['Sex'].astype(str)

# Limpiar columnas innecesarias
df['Categoria_edad'] = pd.cut(df['Age'], bins=[0, 12, 18, 65, 100], labels=['Niño', 'Adolescente', 'Adulto', 'Anciano']).map({'Niño': 4, 'Adolescente': 4, 'Adulto': 2, 'Anciano': 1})
df.drop(['Name', 'Ticket', 'Cabin', 'SibSp', 'Parch', 'Sex', 'Pclass', 'Embarked', 'WikiId', 'Name_wiki', 'Age_wiki', 'Hometown', 'Boarded', 'Destination', 'Lifeboat', 'Body', 'Class'], axis=1, inplace=True)

df      #reviso que el formato de los datos me quede igual a como deberia quedar (forma en la que entrene al modelo)


,PassengerId,Age,Fare,Familia,Cuota_por_pasajero,pclass,Titulo,Pclass_Sex,Categoria_edad
0,892,34.500000,7.8292,1,7.829200,1,1,30,2
1,893,47.000000,7.0000,2,3.500000,1,2,31,2
2,894,62.000000,9.6875,1,9.687500,2,1,20,2
3,895,27.000000,8.6625,1,8.662500,1,1,30,2
4,896,22.000000,12.2875,3,4.095833,1,2,31,2
...,...,...,...,...,...,...,...,...,...
413,1305,26.655844,8.0500,1,8.050000,1,1,30,2
414,1306,39.000000,108.9000,1,108.900000,3,0,11,2
415,1307,38.500000,7.2500,1,7.250000,1,1,30,2
416,1308,26.655844,8.0500,1,8.050000,1,1,30,2


In [71]:
#reordeno las columnas para que tenga el mismo orden que en el df de entrenamiento
df = df[['PassengerId', 'Age', 'Fare', 'Familia', 'Cuota_por_pasajero', 'Titulo', 'Pclass_Sex', 'Categoria_edad']]
df

,PassengerId,Age,Fare,Familia,Cuota_por_pasajero,Titulo,Pclass_Sex,Categoria_edad
0,892,34.500000,7.8292,1,7.829200,1,30,2
1,893,47.000000,7.0000,2,3.500000,2,31,2
2,894,62.000000,9.6875,1,9.687500,1,20,2
3,895,27.000000,8.6625,1,8.662500,1,30,2
4,896,22.000000,12.2875,3,4.095833,2,31,2
...,...,...,...,...,...,...,...,...
413,1305,26.655844,8.0500,1,8.050000,1,30,2
414,1306,39.000000,108.9000,1,108.900000,0,11,2
415,1307,38.500000,7.2500,1,7.250000,1,30,2
416,1308,26.655844,8.0500,1,8.050000,1,30,2


Traer el df para corroborar las predicciones

In [72]:
df_resultados = pd.read_csv('/content/drive/MyDrive/IES/Segundo semestre/Aprendizaje automatico I/Trabajo TITANIC Final/FINALASO TODO SUPER FINAL/y_test')
df_resultados.rename(columns={'survived': 'Survived', 'passengerid':'PassengerId'}, inplace=True)


In [73]:
#separo en x | y
X = df[[ 'Age', 'Fare', 'Familia', 'Cuota_por_pasajero', 'Titulo', 'Pclass_Sex', 'Categoria_edad']]
y = df_resultados['Survived']
y = y[:418]
print(X.shape)
print(y.shape)

(418, 7)
(418,)


#RanfomForest_2 (Modelo del 97%) 🥉

In [74]:
import pickle
#traemos nuestro modelo entrenado
with open('/content/drive/MyDrive/IES/Segundo semestre/Aprendizaje automatico I/IEFI/rf_2.plk', "rb") as file:
    rf_2 = pickle.load(file)

In [75]:
from sklearn.metrics import accuracy_score

In [76]:
predicciones_rf_2 = rf_2.predict(X)
predicciones_rf_2

array([0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1,
       1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0,

In [77]:
comparacion_rf_2 = y.values == predicciones_rf_2
print(comparacion_rf_2)

porcentaje_true_rf_2 = np.mean(comparacion_rf_2) * 100
print(f"Porcentaje de True: {porcentaje_true_rf_2:.2f}%")

[ True False  True  True False False  True False False False False  True
 False  True  True False False False False False  True False False False
  True False  True  True False  True  True  True False False False False
  True False False  True False False False False False  True False  True
  True False False False False False False  True  True  True  True  True
  True False  True False False  True  True False  True  True  True False
  True  True False  True False  True  True  True False  True  True  True
  True  True False False False  True False False  True  True False  True
 False False  True  True False  True False  True False False False  True
 False False False False False  True False  True  True False  True  True
 False  True  True  True False False False  True False False  True  True
 False False  True  True  True  True False  True  True False  True False
  True False False  True  True  True False  True False False False  True
 False  True False  True False False  True False Fa

In [78]:
df = df[['PassengerId', 'Age', 'Fare', 'Familia', 'Cuota_por_pasajero', 'Titulo', 'Pclass_Sex', 'Categoria_edad']]

Este primer modelo es uno entrenado con un aumento significativo en la cantidad de muetras, lo que hice fue aplicar operaciones matematicas simples al set de datos para cambiar los valores de los datos pero no la informacion, asi el modelo no memoriza ni tira siempre que murio porque el 80% murio. Lo cual en el conjunto de validacion me dio 97% de accuracy (probable overfiting) y en el momento de predecir obtuve un resultado bastante mas bajo, corroborando que el modelo estaba overfiteado

#RandomForest_1 (modelo previo a la multiplicacion del dataset)🥇


Este modelo fue entrenado con el set de datos original, es decir sin ningun aumento de muestras, pero si habiendo pasado por la ingenieria de variables que puse mas arriba, habiendo dado en el conjunto de validacion un 86% de accuracy, el cual alcanzo a la hora de la puesta en produccion un total de 73,3%, un resultado emjor

In [79]:
import pickle
# traemos nuestro modelo entrenado
with open('/content/drive/MyDrive/IES/Segundo semestre/Aprendizaje automatico I/IEFI/rf_1.plk', "rb") as file:
    rf_1 = pickle.load(file)

In [80]:

predicciones_rf_1 =  rf_1.predict(X)#mopstrar solo esto y la columna survived del df que si tiene y
comparacion_rf_1 = y.values == predicciones_rf_1
print(comparacion_rf_1)

porcentaje_true_rf_1 = np.mean(comparacion_rf_1) * 100
print(f"Porcentaje de True: {porcentaje_true_rf_1:.2f}%")

[ True False False  True False False  True False False False False  True
 False  True False False False False False False  True False  True False
 False False  True  True False  True  True  True False False False False
  True False False  True False False False False  True  True False  True
  True False False False False  True False  True  True  True  True  True
  True False  True False  True  True  True  True  True  True  True False
  True  True False False False  True  True  True False  True  True  True
  True  True False False False  True False False False  True False  True
  True False  True  True  True  True False  True False False False  True
 False False False False False  True False  True  True False False  True
 False  True  True  True False False False  True  True False  True  True
 False False  True  True  True  True False  True  True False  True False
  True  True False  True  True  True False  True False False False  True
 False  True False  True False False  True False Fa

#RanfomForest_3 (modelo llamado supersaijayin) 🥈


Este modelo fue entrenado con el conjunto de datos orignal pero con un RandomOverSampler aplicado para equilibrar el set de datos y que sean 50% vivieron 50% murieron, para que el modelo no tire por probabilidad que se murio

In [81]:
import pickle
# traemos nuestro modelo entrenado
with open('/content/drive/MyDrive/IES/Segundo semestre/Aprendizaje automatico I/IEFI/rf_3.pkl', "rb") as file:
    rf_3 = pickle.load(file)

In [82]:
prediccion_rf_3 =  rf_3.predict(X)
comparacion_rf_3 = y.values == prediccion_rf_3
print(comparacion_rf_3)

porcentaje_rf_3 = np.mean(comparacion_rf_3) * 100
print(f"Porcentaje de True: {porcentaje_rf_3:.2f}%")

[ True False  True  True False False  True False False False False  True
 False  True False False False False False False  True False False False
 False False  True  True False  True  True  True False False False False
  True False False  True False False False False False  True False  True
  True False False False False  True False  True  True  True  True  True
  True False  True False  True  True  True  True  True  True  True False
  True  True False False False  True  True  True False  True  True  True
  True  True False False False  True False False  True  True False  True
  True False  True  True False  True False  True False False False  True
 False False False False False  True False  True  True False False  True
 False  True  True  True False False False  True False False  True  True
 False False  True  True  True  True False  True  True False  True False
  True  True False  True  True  True False  True False False False  True
 False  True False  True False False  True False Fa

#Final del Trabajo (Titanic conclusiones)

En conclusion, algo que pude notar es que los resultados cambiaron brutalmente dependiendo si para hacer mi prediccion utilizaba el df sin y que si utilizaba el df que si tenia la y. Mi teoria es que el df sin y y el df con y, no tienen los datos ordenados de la misma forma, y por eso cuando predigo con un df que no esta ordenado de exactamente la misma forma que el otro mis resultados bajan brutalmente (+25% de acc). La prediccion que deje fue la hecha con el metodo que vos nos dijiste lucho, es decir, utilizando para predecir el set de datos de prediccion y para corroborar las predicciones solamente la columna "y" del df que si las tenia.

intente hacer intente hacer un codigo (claramente con ayuda de chatgpt y gemini) en el cual me ordenara los resultados por la columna PassengerId, pero creo que no funciono de todas formas.

Te dejo abajo el intento de codigo ya que si lo dejo arriba rompe todo el codigo de abajo y la frustracion no me da para seguir renegando con esto :)

In [83]:
import pandas as pd

# Supongamos que df_resultados contiene "PassengerId" y es el DataFrame con los IDs correspondientes
# predicciones_rf_2 es tu array de predicciones
# y es tu DataFrame de valores reales que también incluye "PassengerId" y la columna de etiqueta real

# Verificar que las longitudes de df_resultados y predicciones_rf_2 coincidan
# Si df_resultados tiene más filas, recortarlo para que coincida con la longitud de predicciones_rf_2
if len(df_resultados) != len(predicciones_rf_2):
    df_resultados = df_resultados.iloc[:len(predicciones_rf_2)]

# Convertir el array de predicciones en una Serie o DataFrame
pred_df = pd.DataFrame({
    'PassengerId': df_resultados['PassengerId'].values,  # Asegura que los IDs están alineados
    'columna_pred': pd.Series(predicciones_rf_2)         # Convertir predicciones_rf_2 en Serie
})

# Check if 'PassengerId' exists in y, and convert to DataFrame if needed
if not isinstance(y, pd.DataFrame) or 'PassengerId' not in y.columns:
    y = pd.DataFrame({'PassengerId': df_resultados['PassengerId'].values, 'columna_real': y.values})

# Combinar ambos DataFrames en base a "PassengerId"
merged_df = pd.merge(pred_df, y, on="PassengerId", suffixes=('_pred', '_real'))

# Ordenar el DataFrame resultante por "PassengerId" para asegurar la alineación correcta
merged_df = merged_df.sort_values(by="PassengerId").reset_index(drop=True)

# Separar las columnas de predicción y reales si es necesario
predictions_aligned = merged_df['columna_pred']  # Cambia "columna_pred" si deseas otro nombre para la columna de predicciones
reales_aligned = merged_df['columna_real']       # Cambia "columna_real" por el nombre de tu columna de valores reales en `y`

# Verificar la alineación de las primeras filas
print(merged_df)

     PassengerId  columna_pred  columna_real
0            892             0           0.0
1            892             0           0.0
2            892             1           0.0
3            892             1           0.0
4            893             1           1.0
..           ...           ...           ...
417         1303             0           1.0
418         1304             1           0.0
419         1305             0           0.0
420         1306             0           1.0
421         1307             1           0.0

[422 rows x 3 columns]


In [84]:
import pandas as pd

# Assuming df_resultados and predicciones_rf_2 are already defined

# 1. Create a DataFrame from predicciones_rf_2 with PassengerId
pred_df = pd.DataFrame({
    'PassengerId': df_resultados['PassengerId'].values,  # Use PassengerId from df_resultados
    'Prediction': predicciones_rf_2
})

# 2. Check if 'PassengerId' exists in df_resultados, and convert to DataFrame if needed
if not isinstance(df_resultados, pd.DataFrame) or 'PassengerId' not in df_resultados.columns:
    df_resultados = pd.DataFrame({'PassengerId': df_resultados, 'some_other_column': some_other_data})  # Replace some_other_data if needed

# 3. Merge the two DataFrames based on PassengerId
merged_df = pd.merge(df_resultados, pred_df, on="PassengerId", how="outer", indicator=True)

# 4. Identify mismatched PassengerIds
mismatched_ids = merged_df[merged_df['_merge'] != 'both']['PassengerId']

# 5. Print the results
if mismatched_ids.empty:
    print("All PassengerIds match in both DataFrames.")
else:
    print("Mismatched PassengerIds:")
    print(mismatched_ids)

All PassengerIds match in both DataFrames.
